In [1]:
import $ivy.`org.apache.spark::spark-mllib:2.4.4` // Or use any other 2.x version here
import $ivy.`default::steamrec:0.1.0-SNAPSHOT`


import $ivy.$                                     // Or use any other 2.x version here



In [2]:
import com.steamrec.DataLoader
import com.steamrec.User
import com.steamrec.Game
import org.apache.spark.ml.recommendation.ALS.Rating
import org.apache.spark.ml.recommendation.ALSModel
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import $ivy.$                                 

import org.apache.spark.ml.recommendation.ALSModel

import org.apache.log4j.{Level, Logger}


In [4]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}
import spark.implicits._

Loading spark-stubs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@2fcfa103
import spark.implicits._

In [5]:
val model_path = "data_dir/rank_500_it_10_lambda_0.01_alpha_10.0"

model_path: String = "data_dir/rank_500_it_10_lambda_0.01_alpha_10.0"

In [6]:
val model = ALSModel.load(s"${model_path}/model")

first at ReadWrite.scala:615

1 / 1

load at ALS.scala:527

1 / 1

load at ALS.scala:529

1 / 1

model: ALSModel = als_f54b54223746

import com.steamrec.DataLoader

import com.steamrec.User

import com.steamrec.Game

import org.apache.spark.ml.recommendation.ALS.Rating


In [8]:
val data = DataLoader.loadData("data_dir/train-test-split-v2.json")

json at DataLoader.scala:11

9 / 9

collect at GameData.scala:15

9 / 9

data: com.steamrec.GameData = com.steamrec.GameData@192c9522

In [9]:
data.dataSet.head(1)

head at cmd8.sc:1

1 / 1

res8: Array[User] = Array(
  User(
    0,
    List(
      Game(0, "Dota 2 Test", 0),
      Game(1, "Gear Up", 1),
      Game(2, "Left 4 Dead 2", 9),
      Game(3, "Counter-Strike: Global Offensive", 2493),
      Game(4, "SMITE", 540),
      Game(5, "Smite - Public Test", 0),
      Game(6, "Paladins", 49656),
      Game(7, "Paladins - Public Test", 0),
      Game(8, "Team Fortress 2", 1086),
      Game(9, "Dota 2", 71638)
    ),
    List(
      Game(0, "Dota 2 Test", 0),
      Game(3, "Counter-Strike: Global Offensive", 2493),
      Game(4, "SMITE", 540),
      Game(5, "Smite - Public Test", 0),
      Game(6, "Paladins", 49656),
      Game(7, "Paladins - Public Test", 0),
      Game(8, "Team Fortress 2", 1086),
      Game(9, "Dota 2", 71638)
    ),
    List(Game(2, "Left 4 Dead 2", 9), Game(1, "Gear Up", 1))
  )
)

In [10]:
import com.steamrec.ModelUtils

import com.steamrec.ModelUtils

In [11]:
val userSubset = spark.createDataset(Seq(0, 1, 2, 3,4,5,6,7,8,9)).toDF("user")

userSubset: DataFrame = [user: int]

In [18]:
val ratings = model.recommendForAllUsers(100)

ratings: DataFrame = [user: int, recommendations: array<struct<item:int,rating:float>>]

In [13]:

import scala.collection.mutable
case class RatingItem(item: Int, rating: Float)
case class GroupedRating(user: Int, recommendations: Array[RatingItem])

def rr(user: User, ratings: Iterator[Rating[Int]]): Double = {
    val trainGames = user.train_games.map(g => g.game_id).toSet
    val validGames = user.valid_games.map(g => g.game_id).toSet
    val sortedRatings = ratings.toList
      .filter(x => !trainGames.contains(x.item))
      .sortBy(r => -r.rating);
    sortedRatings.zipWithIndex.find((r)=>  validGames contains r._1.item)
      .map(r => 1.0/ (r._2 + 1))
      .getOrElse(0.0)


}

  def mapItem(uid:Int, ratingItem:Row): Rating[Int] = {
    Rating(uid, ratingItem.getAs[Int]("item"), ratingItem.getAs[Float]("rating"))
  }

  def row2ratings(row: Row): Iterator[Rating[Int]] = {
    val uid = row.getAs[Int](0)
    row.getAs[mutable.WrappedArray[Row]]("recommendations")
      .map(x => mapItem(uid, x)).toIterator
  }

import scala.collection.mutable

defined class RatingItem
defined class GroupedRating
defined function rr
defined function mapItem
defined function row2ratings

In [39]:
val uRatings = ratings.filter(r=> r.getAs[Int](0)==148).head.getAs[mutable.WrappedArray[(Int, Float)]](1)

head at cmd38.sc:1

50 / 50

head at cmd38.sc:1

1 / 1

uRatings: mutable.WrappedArray[(Int, Float)] = WrappedArray(
  [6,1.0716201],
  [945,1.00699],
  [683,1.000523],
  [3,0.9999968],
  [131,0.99995977],
  [297,0.9999597],
  [1307,0.99993116],
  [8,0.9998051],
  [1915,0.9994115],
  [265,0.9982012],
  [279,0.994965],
  [1392,0.9868357],
  [951,0.9864615],
  [2516,0.9817046],
  [682,0.930246],
  [1821,0.8870774],
  [164,0.8730698],
  [1163,0.87027],
  [2076,0.8442739],
  [1914,0.8424841],
  [2434,0.83562386],
  [586,0.82029873],
  [42,0.81831825],
  [1308,0.72562504],
  [7,0.70215976],
  [585,0.6607297],
  [952,0.6388063],
  [280,0.63654745],
  [1795,0.5890172],
  [898,0.53465796],
  [1800,0.47093648],
  [132,0.43330297],
  [2518,0.40926215],
  [953,0.37479985],
  [21,0.35347968],
  [66,0.3431212],
  [1417,0.3313372],
  [1077,0.33130622],
...

In [41]:
data.id2User.value(148).valid_games

res40: List[Game] = List(
  Game(262, "War Thunder", 0),
  Game(589, "Tom Clancy's Ghost Recon Phantoms - EU", 42),
  Game(44, "Oddworld: Abe's Oddysee", 0),
  Game(186, "Dirty Bomb", 45),
  Game(66, "Warface", 52),
  Game(955, "Source Filmmaker", 31),
  Game(1002, "Pro Evolution Soccer 2016 myClub", 8),
  Game(1140, "Tom Clancy's The Division", 341),
  Game(1141, "Tom Clancy's The Division PTS", 0)
)

In [16]:
val rrs: Dataset[(Int, Double)] = ratings.map(r => (r.getAs[Int]("user"),rr(data.id2User.value(r.getAs("user")), row2ratings(r))))

rrs: Dataset[(Int, Double)] = [_1: int, _2: double]

In [42]:
rrs.collect.map(x=>x._2).sum / rrs.count

collect at cmd41.sc:1

50 / 50

collect at cmd41.sc:1

200 / 200

count at cmd41.sc:1

...

count at cmd41.sc:1

200 / 200

count at cmd41.sc:1

1 / 1

res41: Double = 0.6315159835655693